In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_selection import mutual_info_regression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin_min
from scipy.stats import ttest_ind
from sklearn.ensemble import RandomForestRegressor


In [ ]:
# Load the CSV file (no header)
df = pd.read_csv("USCensus1990.csv", header=None)

# Define column names (caseid + 68 attributes)
attribute_names = [
    "dAge", "dAncstry1", "dAncstry2", "iAvail", "iCitizen", "iClass", "dDepart", "iDisabl1", "iDisabl2", "iEnglish",
    "iFeb55", "iFertil", "dHispanic", "dHour89", "dHours", "iImmigr", "dIncome1", "dIncome2", "dIncome3", "dIncome4",
    "dIncome5", "dIncome6", "dIncome7", "dIncome8", "dIndustry", "iKorean", "iLang1", "iLooking", "iMarital",
    "iMay75880", "iMeans", "iMilitary", "iMobility", "iMobillim", "dOccup", "iOthrserv", "iPerscare", "dPOB",
    "dPoverty", "dPwgt1", "iRagechld", "dRearning", "iRelat1", "iRelat2", "iRemplpar", "iRiders", "iRlabor",
    "iRownchld", "dRpincome", "iRPOB", "iRrelchld", "iRspouse", "iRvetserv", "iSchool", "iSept80", "iSex", "iSubfam1",
    "iSubfam2", "iTmpabsnt", "dTravtime", "iVietnam", "dWeek89", "iWork89", "iWorklwk", "iWWII", "iYearsch",
    "iYearwrk", "dYrsserv"
]
df.columns = ['caseid'] + attribute_names


In [ ]:
# Remove extreme outlier 
df = df[df['dHours'] != 15]

In [ ]:
# Remove 'caseid' 
df = df.drop(columns=['caseid'])

# Remove highly correlated columns
X = df.drop(columns=['dHours', 'dIncome1', 'dHour89', 'dRearning'], errors='ignore')
y = df['dHours']

# Verify non-numerical columns
non_numeric = X.select_dtypes(exclude=['number']).columns
print("Non-numerical columns:", non_numeric.tolist())

# Remove non-numerical columns
X = X.drop(columns=non_numeric)

# Fill NAs
X = X.fillna(X.median(numeric_only=True))

# Scale
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=123)

# Define the model
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)


In [ ]:
# Predict 10 random examples from the test set

predictions = model.predict(X_test[:10])
for i in range(10):
    print(f"Predicted: {predictions[i][0]:.1f} | Actual: {y_test.iloc[i]}")


In [ ]:
# Subsample (reduces memory and CPU use)
X_sub = X.iloc[:2000, : ]
y_sub = y.iloc[:2000]

# Compute MI
mi_scores = mutual_info_regression(X_sub, y_sub, discrete_features='auto')
mi_series = pd.Series(mi_scores, index=X_sub.columns).sort_values(ascending=False)

# Show top 5
top_features = mi_series.head(5)
for i, (feature, score) in enumerate(top_features.items(), 1):
    print(f"{i}. {feature} — MI Score: {score:.4f}")


In [ ]:
# Subsample still (e.g., 2000 rows), but use all columns
X_sub = X.iloc[:2000, :]
y_sub = y.iloc[:2000]

# Compute Mutual Information
mi_scores = mutual_info_regression(X_sub, y_sub, discrete_features='auto')
mi_series = pd.Series(mi_scores, index=X_sub.columns).sort_values(ascending=False)

# Show top 10
top_features = mi_series.head(10)
top_features.plot(kind='bar', title='Top 10 Features by Mutual Information', ylabel='MI Score')


Reduce to top features

In [ ]:
# Use only the top N features from MI
top_N = 10
selected_features = mi_series.head(top_N).index.tolist()

# Redefine X with only those features
X_reduced = X[selected_features]

# Normalization again (Min-Max)
scaler = MinMaxScaler()
X_scaled_reduced = scaler.fit_transform(X_reduced)

# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled_reduced, y, test_size=0.2, random_state=123)


In [ ]:
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)


In [ ]:
# Create treatment variable (binary)
df['treated'] = (df['dHours'] <= 2).astype(int)

# Choose covariates (ex: top 10 MI features)
covariates = selected_features  # from MI ranking
X_covariates = df[covariates]
X_covariates = X_covariates.fillna(X_covariates.median(numeric_only=True))

# Target outcome to evaluate (e.g., income or other)
outcome = 'dIncome1'
y_outcome = df[outcome]


In [ ]:
print(df['dHours'].describe())
print(df['treated'].value_counts())


In [ ]:
# Fit logistic regression to estimate propensity scores
ps_model = LogisticRegression(max_iter=1000)
ps_model.fit(X_covariates, df['treated'])

# Get propensity scores
df['propensity_score'] = ps_model.predict_proba(X_covariates)[:, 1]


In [ ]:
# Split treated and control
treated = df[df['treated'] == 1]
control = df[df['treated'] == 0]

# Fit nearest neighbors on control group
nn = NearestNeighbors(n_neighbors=1)
nn.fit(control[['propensity_score']])

# Find nearest neighbor for each treated unit
distances, indices = nn.kneighbors(treated[['propensity_score']])
matched_control = control.iloc[indices.flatten()]

# Compute ATE
ate = (treated[outcome].reset_index(drop=True) - matched_control[outcome].reset_index(drop=True)).mean()
print(f"Estimated ATE (dHours <= 2 on {outcome}): {ate:.3f}")


In [ ]:
# 1. Define covariates, treatment, and outcome
X = df[selected_features].copy()
y = df["treated"]
outcome = df["dIncome1"]

# 2. Handle missing values and scale features
X = X.fillna(X.median(numeric_only=True))
X_scaled = MinMaxScaler().fit_transform(X)

# 3. Train Random Forest classifier to estimate propensity scores
rf = RandomForestClassifier(n_estimators=100, random_state=123)
rf.fit(X_scaled, y)
propensity_scores = rf.predict_proba(X_scaled)[:, 1]

# 4. Add propensity scores to the DataFrame
df_psm = df.copy()
df_psm["propensity_score"] = propensity_scores
df_psm["dIncome1"] = outcome
df_psm["treated"] = y.values

# 5. Separate treated and control groups
treated_df = df_psm[df_psm["treated"] == 1].copy()
control_df = df_psm[df_psm["treated"] == 0].copy()

# 6. Match each treated unit to the nearest control based on propensity score
nn = NearestNeighbors(n_neighbors=1, algorithm="ball_tree").fit(control_df[["propensity_score"]])
distances, indices = nn.kneighbors(treated_df[["propensity_score"]])
matched_controls = control_df.iloc[indices.flatten()].copy()

# 7. Estimate the Average Treatment Effect (ATE)
treated_outcomes = treated_df["dIncome1"].values
control_outcomes = matched_controls["dIncome1"].values
ate = np.mean(treated_outcomes - control_outcomes)

print(f"Estimated ATE (treated vs. matched control on dIncome1): {ate:.3f}")


In [ ]:
# Estimate CATE (Conditional Average Treatment Effect) by subgroup using correct column names

# Create age groups based on dAge
df['age_group'] = pd.cut(df['dAge'], bins=[0, 25, 40, 60, 100], labels=['<=25', '26-40', '41-60', '60+'])

# Define subgroup columns based on your dataset
subgroups = {
    'Sex': 'iSex',
    'Education': 'iSchool',
    'Age Group': 'age_group'
}

results = []

for name, col in subgroups.items():
    for level in df[col].dropna().unique():
        df_sub = df[df[col] == level].copy()

        if df_sub['treated'].nunique() < 2:
            continue

        covariates = df_sub.drop(columns=['treated', 'dIncome1', 'dHours', 'dHour89', 'dRearning', 'dWeek89', 'propensity_score'], errors='ignore')
        covariates = covariates.select_dtypes(include=[np.number]).fillna(0)

        if covariates.shape[1] == 0:
            continue

        ps_model = LogisticRegression(max_iter=1000)
        ps_model.fit(covariates, df_sub['treated'])
        df_sub['propensity_score'] = ps_model.predict_proba(covariates)[:, 1]

        treated = df_sub[df_sub['treated'] == 1]
        control = df_sub[df_sub['treated'] == 0]

        if len(treated) == 0 or len(control) == 0:
            continue

        nn = NearestNeighbors(n_neighbors=1)
        nn.fit(control[['propensity_score']])
        _, indices = nn.kneighbors(treated[['propensity_score']])
        matched_controls = control.iloc[indices.flatten()]

        ate = (treated['dIncome1'].values - matched_controls['dIncome1'].values).mean()

        results.append({
            'Subgroup': name,
            'Level': level,
            'CATE': ate,
            'N Treated': len(treated),
            'N Control': len(control)
        })

cate_df = pd.DataFrame(results)
display(cate_df)



In [ ]:


# Map readable names to actual DataFrame column names
subgroup_map = {
    'Sex': 'iSex',
    'Education': 'iSchool',
    'Age Group': 'age_group'
}

# Ensure 'age_group' is a string if it exists
if 'age_group' in df.columns:
    df['age_group'] = df['age_group'].astype(str)

# Define CATE results (pre-calculated)
cate_df = pd.DataFrame({
    'Subgroup': ['Sex', 'Sex', 'Education', 'Education', 'Education', 'Age Group'],
    'Level': [1, 0, 1, 2, 3, '<=25'],
    'CATE': [-0.392255, -0.141909, 0.942493, -0.691051, 0.459430, -0.270070],
    'N Treated': [11801, 9337, 13981, 5140, 912, 20802],
    'N Control': [997, 2448, 3204, 179, 62, 3446]
})

# Perform Welch’s t-test for each subgroup
p_values = []
for _, row in cate_df.iterrows():
    subgroup = row['Subgroup']
    level = row['Level']
    col = subgroup_map[subgroup]
    level = str(level) if col == 'age_group' else int(level)

    treated_vals = df[(df['treated'] == 1) & (df[col] == level)]['dIncome1']
    control_vals = df[(df['treated'] == 0) & (df[col] == level)]['dIncome1']

    t_stat, p_val = ttest_ind(treated_vals, control_vals, equal_var=False, nan_policy='omit')
    p_values.append(p_val)

# Add results to CATE DataFrame
cate_df['p-value'] = p_values
cate_df['Significant (p < 0.05)'] = cate_df['p-value'] < 0.05

# show table
display(cate_df)



In [ ]:


s
X = df[selected_features].copy()
y = df["dIncome1"]

# Preencher valores em falta
X = X.fillna(X.median(numeric_only=True))

# scale
X_scaled = MinMaxScaler().fit_transform(X)

# Train model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_scaled, y)

# Importância das features
importances = rf_regressor.feature_importances_
feature_names = X.columns

# dataframe of importance
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plot
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
plt.title('Feature Importance from Random Forest Regressor')
plt.xlabel('Relative Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

In [ ]:


# Prepare groups
df['age_group'] = pd.cut(df['dAge'], bins=[0, 25, 40, 60, 100], labels=['<=25', '26-40', '41-60', '60+'])
subgroups = {'Sex': 'iSex', 'Education': 'iSchool', 'Age Group': 'age_group'}

results = []

for name, col in subgroups.items():
    for level in df[col].dropna().unique():
        df_sub = df[df[col] == level].copy()
        if df_sub['treated'].nunique() < 2:
            continue

        # Covariates para modelo de propensão
        covariates = df_sub.drop(columns=['treated', 'dIncome1', 'dHours', 'dHour89', 'dRearning', 'dWeek89'], errors='ignore')
        covariates = covariates.select_dtypes(include=[np.number]).fillna(0)

        if covariates.shape[1] == 0:
            continue

        # estimate propensity score
        ps_model = LogisticRegression(max_iter=5000)
        ps_model.fit(covariates, df_sub['treated'])
        df_sub['propensity_score'] = ps_model.predict_proba(covariates)[:, 1]

        treated = df_sub[df_sub['treated'] == 1]
        control = df_sub[df_sub['treated'] == 0]

        if len(treated) == 0 or len(control) == 0:
            continue

        nn = NearestNeighbors(n_neighbors=1)
        nn.fit(control[['propensity_score']])
        _, indices = nn.kneighbors(treated[['propensity_score']])
        matched_controls = control.iloc[indices.flatten()].copy()

        cate = (treated['dIncome1'].values - matched_controls['dIncome1'].values).mean()

        # Welch t-test 
        t_vals = df[(df['treated'] == 1) & (df[col] == level)]['dIncome1']
        c_vals = df[(df['treated'] == 0) & (df[col] == level)]['dIncome1']
        t_stat, p_val = ttest_ind(t_vals, c_vals, equal_var=False, nan_policy='omit')

        results.append({
            'Subgroup': name,
            'Level': level,
            'CATE': cate,
            'N Treated': len(treated),
            'N Control': len(control),
            'p-value': p_val,
            'Significant (p < 0.05)': p_val < 0.05
        })

# create dataframe
cate_df = pd.DataFrame(results)
cate_df = cate_df.sort_values(by=['Subgroup', 'Level']).reset_index(drop=True)

# show on notebook
styled_table = cate_df.style.format({
    'CATE': '{:.3f}',
    'p-value': '{:.4e}'
}).background_gradient(subset=['CATE'], cmap='coolwarm').highlight_null()

display(styled_table)


In [ ]:


features_to_check = selected_features

# Before do matching
df['group'] = df['treated']
scaler = StandardScaler()
smds_before = []

for feature in features_to_check:
    treated_vals = df[df['group'] == 1][feature].dropna()
    control_vals = df[df['group'] == 0][feature].dropna()

    smd = abs(treated_vals.mean() - control_vals.mean()) / np.sqrt((treated_vals.std()**2 + control_vals.std()**2) / 2)
    smds_before.append(smd)

# After matching


treated = df[df['treated'] == 1].copy()
control = df[df['treated'] == 0].copy()

nn = NearestNeighbors(n_neighbors=1)
nn.fit(control[['propensity_score']])
_, indices = nn.kneighbors(treated[['propensity_score']])
matched_control = control.iloc[indices.flatten()].copy()

smds_after = []

for feature in features_to_check:
    treated_vals = treated[feature].dropna()
    control_vals = matched_control[feature].dropna()

    smd = abs(treated_vals.mean() - control_vals.mean()) / np.sqrt((treated_vals.std()**2 + control_vals.std()**2) / 2)
    smds_after.append(smd)

# Create table
smd_df = pd.DataFrame({
    'Feature': features_to_check,
    'SMD Before Matching': smds_before,
    'SMD After Matching': smds_after
})

# Show table
print("\nTable – Standardized Mean Differences Before and After Matching")
display(smd_df.round(3))




In [ ]:
plt.figure(figsize=(12, 6))

# Before matching
plt.subplot(1, 2, 1)
sns.kdeplot(df[df['treated'] == 1]['propensity_score'], label='Treated', fill=True, alpha=0.5)
sns.kdeplot(df[df['treated'] == 0]['propensity_score'], label='Control', fill=True, alpha=0.5)
plt.title("Before Matching")
plt.xlabel("Propensity Score")
plt.legend()

# After matching
plt.subplot(1, 2, 2)
sns.kdeplot(treated['propensity_score'], label='Treated', fill=True, alpha=0.5)
sns.kdeplot(matched_control['propensity_score'], label='Matched Control', fill=True, alpha=0.5)
plt.title("After Matching")
plt.xlabel("Propensity Score")
plt.legend()

plt.tight_layout()
plt.suptitle("Figure 3.2 – Propensity Score Distribution Before and After Matching", y=1.05)
plt.savefig("Figure_3_2_Propensity_Matching.png", bbox_inches='tight')
plt.show()
